In [1]:
import pandas as pd
import numpy as np
import os
import pandas as pd
import rasterio
import numpy as np
from rasterio.mask import mask
from shapely.geometry import mapping

from vayu_gnn.dbx.dbx_config import dbx_helper, DropboxHelper

In [2]:
def pixel_sum_around_point(gdf, raster_bytes, stat_col_name, buffer_distance=0):
    """
    Returns a DataFrame with each device's identifier and its corresponding raster summary statistic.
    
    For single-band rasters, when buffer_distance is 0, the raster value is sampled exactly at the point.
    If buffer_distance > 0, a buffer polygon is created around the point, pixels within that area are extracted,
    and their values are summed.
    
    Args:
        gdf (GeoDataFrame): A GeoDataFrame containing device geometries (Points) and an optional
                            'device_id' column. If missing, the index is used.
        raster_bytes (BytesIO): The in-memory TIFF file (assumed to be single-band).
        buffer_distance (float): The buffer distance (in CRS units). Set to 0 to sample the exact point.
    
    Returns:
        DataFrame: A DataFrame with columns 'device_id' and 'summary_stat', where summary_stat is a scalar.
    """
    results = {}
    
    with rasterio.open(raster_bytes) as src:
        raster_crs = src.crs
        
        # Reproject GeoDataFrame if its CRS doesn't match the raster's CRS.
        if gdf.crs != raster_crs:
            print(f"Reprojecting GeoDataFrame from {gdf.crs} to {raster_crs}")
            devices_in_raster_crs = gdf.to_crs(raster_crs)
        else:
            devices_in_raster_crs = gdf
        
        for idx, row in devices_in_raster_crs.iterrows():
            device_id = row.get("device_id", idx)
            geom = row.geometry
            
            if buffer_distance == 0:
                # Sample the raster value at the exact point location.
                for val in src.sample([(geom.x, geom.y)]):
                    results[device_id] = float(val[0])
            else:
                # Create a buffer polygon around the point.
                buffer_geom = geom.buffer(buffer_distance)
                geom_mapping = [mapping(buffer_geom)]
                
                # Mask the raster to get pixels within the buffer.
                out_image, _ = mask(src, geom_mapping, crop=True)
                
                # Handle nodata values by converting them to NaN.
                nodata = src.nodata
                if nodata is not None:
                    out_image = np.where(out_image == nodata, np.nan, out_image)
                # For a single band, out_image shape is (1, height, width).
                band_sum = np.nansum(out_image[0])
                results[device_id] = float(band_sum)
    
    # Convert the results into a DataFrame.
    data = [{"device_id": device_id, stat_col_name: summary} for device_id, summary in results.items()]
    return pd.DataFrame(data)

In [6]:
cities = ['Patna']

for city in cities:

    devices = dbx_helper.read_shp(dbx_helper.clean_input_path, f'node_locations/{city}/gdf')

    raster_files = ['elevation', 
                    'GHS_total_building_area',
                    'GHS_total_building_volume',
                    'GHS_non_res_building_area',
                    'GHS_non_res_building_volume']

    for file in raster_files:

        if file == 'elevation':
            tif = dbx_helper.read_tif(dbx_helper.raw_input_path, 'elevation', 'elevation.tif')
        
            print(f'Calculating {file}')
            result = pixel_sum_around_point(devices, tif, 'elevation', buffer_distance=0)
            dbx_helper.write_csv(result, dbx_helper.clean_input_path, f'{file}/{city}', f'{file}.csv')

        else:

            tif = dbx_helper.read_tif(dbx_helper.raw_input_path, 'settlement', f'{file}.tif')
            
            distances = [0,500]
            for buffer in distances: 
                print(f'Calculating {file} buffer {buffer}')
    
                stat_col_name = 'sum_' + file + '_buffer_' + str(buffer)
                result = pixel_sum_around_point(devices, tif, stat_col_name, buffer_distance=buffer)
                dbx_helper.write_csv(result, dbx_helper.clean_input_path, f'settlement/{city}', f'{file}_b{buffer}.csv')
    

Calculating elevation
Size of the CSV file: 0.00 MB
File 'elevation.csv' successfully uploaded to Dropbox path: '/input/clean/elevation/Patna/elevation.csv'
Calculating GHS_total_building_area buffer 0
Reprojecting GeoDataFrame from EPSG:4326 to ESRI:54009
Size of the CSV file: 0.00 MB
File 'GHS_total_building_area_b0.csv' successfully uploaded to Dropbox path: '/input/clean/settlement/Patna/GHS_total_building_area_b0.csv'
Calculating GHS_total_building_area buffer 500
Reprojecting GeoDataFrame from EPSG:4326 to ESRI:54009
Size of the CSV file: 0.00 MB
File 'GHS_total_building_area_b500.csv' successfully uploaded to Dropbox path: '/input/clean/settlement/Patna/GHS_total_building_area_b500.csv'
Calculating GHS_total_building_volume buffer 0
Reprojecting GeoDataFrame from EPSG:4326 to ESRI:54009
Size of the CSV file: 0.00 MB
File 'GHS_total_building_volume_b0.csv' successfully uploaded to Dropbox path: '/input/clean/settlement/Patna/GHS_total_building_volume_b0.csv'
Calculating GHS_total